# Connect to ClickHouse Database

In [37]:
!mkdir -p /home/jupyter/datasphere/project/.clickhouse && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document /home/jupyter/datasphere/project/.clickhouse/root.crt && \
chmod 0600 /home/jupyter/datasphere/project/.clickhouse/root.crt

--2024-09-08 17:32:15--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’

/home/jupyter/datas 100%[===================>]   3.50K  --.-KB/s    in 0s      

2024-09-08 17:32:16 (840 MB/s) - ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’ saved [3579/3579]



In [38]:
# указываем параметры подключения
CA = '/home/jupyter/datasphere/project/.clickhouse/root.crt' # если вы не меняли путь при скачивании сертификата

VERIFY = True
SECURE = True
USER = 'admin'
PASSWORD = 'hakaton1'
HOST = "rc1d-uq593a42e02ikh2k.mdb.yandexcloud.net" # адрес Clickhouse
PORT = 8443

In [39]:
!pip install clickhouse_connect > /dev/null

In [40]:
# импортируем библиотеку
import clickhouse_connect

# выполняем запрос - в случае успешного выполнения будет показана текущая версия Clickhouse
with clickhouse_connect.get_client(
            host=HOST, port=PORT, username=USER,
            password=PASSWORD, secure=SECURE, verify=VERIFY, ca_cert=CA) as ch_client:
        print(ch_client.command('SELECT version()'))

24.3.10.33


# Web Scraping

In [5]:
!pip install selenium chromedriver_autoinstaller beautifulsoup4 tqdm > /dev/null

In [7]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

In [8]:
driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(10)

In [9]:
from bs4 import BeautifulSoup

url = 'https://nlmk-group.github.io/ds-2.0/?path=/docs/components-sidebar-info--docs'

driver.get(url)

soup = BeautifulSoup(driver.page_source, 'lxml')
# print(soup.prettify())

components = list(map(lambda c: c.get_text(), soup.findAll('button', {"data-parent-id": "components"})))
components[:5]

['Accordion', 'Alert', 'AttachFiles', 'Avatar', 'Badge']

In [10]:
docs = {}
completed_set = set()

In [13]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm
import time
import random

sleep_time = [2, 3, 5, 7, 11]

for c_index, c in tqdm(enumerate(components), total=len(components)):
    if c_index in completed_set: continue

    url_template = 'https://nlmk-group.github.io/ds-2.0/?path=/docs/components-{}-info--docs'
    url = url_template.format(c.lower())

    time.sleep(random.choice(sleep_time))

    driver.get(url)

    iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//iframe[@id='storybook-preview-iframe']")))
    driver.switch_to.frame(iframe)

    # Parse the HTML
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # print(soup.prettify())

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//span[@data-testid='typography-wrapper']"))
    )

    spans = soup.findAll('span', {'data-testid': 'typography-wrapper', 'class': '_typography--variant-Body1_1lcvd_131 _typography_1lcvd_1'})

    spans_text = list(map(lambda c: c.get_text(), spans))

    # print(f"{len(spans_text)=}")
    # print('\n'.join(spans_text))
    # print()

    description = spans_text[0]
    spans_text = spans_text[1:]

    code = soup.findAll('div', {'class': 'sp-code-editor'})
    codes_text = [div.find('pre').get_text() for div in code]

    completed_set.add(c_index)

    # print(f"{len(codes_text)=}")
    # print(codes_text[0])
    # print('-' * 100)

    for d_index, data in enumerate(zip(spans_text, codes_text)):
        key = f'{c}:{c_index}:{d_index}'
        if key not in docs:
            value = f"{c}\n{description}\n{data[0]}\n{data[1]}"
            docs[key] = value

100%|██████████| 44/44 [04:59<00:00,  6.81s/it]


In [14]:
len(docs)

164

In [15]:
list(docs.items())[50]

('DatePicker:11:3',
 "DatePicker\nКомпонент для выбора даты и времени, с возможностью ограничения времени, выбором периода времени и опциональным сдвигом.\nВозможно установить ограничение выбора дат в определенном диапазоне с помощью пропсов с приставкой enabled\nimport { useState } from 'react';\nimport { DatePicker } from '@nlmk/ds-2.0';\n\nconst App = () => {\n  const [value, setValue] = useState(new Date());\n  const today = new Date();\n\n  return (\n    <DatePicker value={value} onChange={setValue} enabledFrom={new Date(today.getFullYear(), 0, 1)} enabledTo={new Date(today.getFullYear(), 11, 31)} />\n  );\n};\nexport default App;\n\n")

In [16]:
driver.quit()

# IAM токен для эмбеддингов

In [20]:
# Устанавливаем пакеты, необходимые для обращения к YandexGPT API
%pip uninstall jwt
%pip install PyJWT -U

In [21]:
import requests
import json
import time
import jwt
import os

In [22]:
# Замените <идентификатор_сервисного_аккаунта> на ваше значение
service_account_id = "aje286em9u4ibs8572di"

In [24]:
with open('/content/authorized_key.json') as f:
    authorized_key = json.load(f)

In [25]:
# Замените <идентификатор_ключа> на ваше значение
key_id = "ajeo9id7551hpq15fv3o"
private_key = authorized_key['private_key']

In [26]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
iam_token = x['iamToken']

# Инициализация базы векторных представлений

In [17]:
!pip install -q langchain_community langchain yandexcloud > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [27]:
# llm
folder = 'b1g01stbeeo96j8kfaiv'
model_uri = f'gpt://{folder}/yandexgpt-lite/latest'
model_uri

'gpt://b1g01stbeeo96j8kfaiv/yandexgpt-lite/latest'

In [41]:
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Cоздаем объект YandexGPTEmbeddings для построения векторов с помощью YandexGPT
embeddings = YandexGPTEmbeddings(iam_token=iam_token, folder_id=folder)

In [42]:
from langchain_community.vectorstores import Clickhouse, ClickhouseSettings

ch_config = ClickhouseSettings(host=HOST, port=PORT, username=USER, password=PASSWORD)
docsearch = Clickhouse(embedding=embeddings, config=ch_config, verify=VERIFY, ca_cert=CA)
docsearch

default.langchain @ rc1d-uq593a42e02ikh2k.mdb.yandexcloud.net:8443

username: admin

Table Schema:
---------------------------------------------------
|id                      |Nullable(String)        |
|document                |Nullable(String)        |
|embedding               |Array(Float32)          |
|metadata                |Object('json')          |
|uuid                    |UUID                    |
---------------------------------------------------

# Очистить базу

In [36]:
docsearch.drop()

# Заполнить базу

In [43]:
from langchain_core.documents import Document

documents = [Document(page_content=v) for k, v in docs.items()]
ids = [k for k, v in docs.items()]

ids[:2], documents[:2]

(['Accordion:0:0', 'Accordion:0:1'],
 [Document(page_content='Accordion\nКомпонент "аккордеон" предоставляет большие объемы контента в ограниченном пространстве с помощью пошагового раскрытия. Заголовок обеспечивает пользователю общий обзор содержимого, позволяя решить, какие разделы читать.\n\n        Аккордеоны могут сделать обработку информации и поиск более эффективными. Однако они скрывают контент от пользователей, и важно учесть, что пользователь может не заметить или не прочитать весь включенный контент. Если пользователь, вероятно, должен прочесть весь контент, не используйте аккордеон, так как это добавляет дополнительный клик; вместо этого используйте полноценную прокручиваемую страницу с обычными заголовками.\nАккордеон по умолчанию\nimport { Accordion } from \'@nlmk/ds-2.0\';\n\nconst items = [\n    {\n      id: 1,\n      title: \'First accordion item\',\n      content: \'Lorem ipsum\',\n      initiallyExpanded: true\n    },\n    {\n      id: 2,\n      title: \'2nd accordio

In [44]:
docsearch.add_documents(documents=documents, ids=ids)

Inserting data...: 100%|██████████| 164/164 [00:02<00:00, 71.22it/s]


['Accordion:0:0',
 'Accordion:0:1',
 'Accordion:0:2',
 'Accordion:0:3',
 'Alert:1:0',
 'Alert:1:1',
 'Alert:1:2',
 'Alert:1:3',
 'AttachFiles:2:0',
 'AttachFiles:2:1',
 'AttachFiles:2:2',
 'AttachFiles:2:3',
 'AttachFiles:2:4',
 'AttachFiles:2:5',
 'AttachFiles:2:6',
 'AttachFiles:2:7',
 'Avatar:3:0',
 'Avatar:3:1',
 'Avatar:3:2',
 'Avatar:3:3',
 'Avatar:3:4',
 'Avatar:3:5',
 'Badge:4:0',
 'Badge:4:1',
 'Badge:4:2',
 'Badge:4:3',
 'Box:5:0',
 'Breadcrumbs:6:0',
 'Breadcrumbs:6:1',
 'Breadcrumbs:6:2',
 'Button:7:0',
 'Button:7:1',
 'Button:7:2',
 'Button:7:3',
 'Button:7:4',
 'Card:8:0',
 'Card:8:1',
 'Checkbox:9:0',
 'Checkbox:9:1',
 'Checkbox:9:2',
 'Checkbox:9:3',
 'Chip:10:0',
 'Chip:10:1',
 'Chip:10:2',
 'Chip:10:3',
 'Chip:10:4',
 'Chip:10:5',
 'DatePicker:11:0',
 'DatePicker:11:1',
 'DatePicker:11:2',
 'DatePicker:11:3',
 'Divider:12:0',
 'Divider:12:1',
 'Divider:12:2',
 'Divider:12:3',
 'Divider:12:4',
 'Divider:12:5',
 'DragAndDrop:13:0',
 'Drawer:14:0',
 'Drawer:14:1',
 'Drop